In [ ]:
import scipy
from scipy.special import j0, jvp
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.metrics import mean_absolute_error

from IPython.display import FileLink

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import imageio.v2 as imageio

In [ ]:
print(tf.config.experimental.list_physical_devices('GPU'))
# tf.debugging.set_log_device_placement(True)

a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)


In [ ]:
NEURON_COUNT = 32
ACTIVATION = 'tanh'
model = keras.Sequential(
    [
        layers.Input(shape=(1,)),
        layers.Dense(NEURON_COUNT, activation=ACTIVATION),
        layers.Dense(NEURON_COUNT, activation=ACTIVATION),
        layers.Dense(NEURON_COUNT, activation=ACTIVATION),
        layers.Dense(1),
    ]
)

model.summary()

In [ ]:
x_bound = tf.Variable([[0.0]], dtype=tf.float32, trainable=True)
# x_phys = tf.Variable(np.reshape(np.linspace(0.1, 20., 100), (100, 1)), dtype=tf.float32, trainable=True)
x_phys = tf.Variable(np.reshape(np.random.randint(0.1,20., 100), (100, 1)), dtype=tf.float32, trainable=True)
x_inn = tf.Variable(np.reshape(np.linspace(0.1, 7.5, 5), (5, 1)), dtype=tf.float32, trainable=True)
y_inn = j0(x_inn)

In [ ]:
!rm -fr checkpoints img
!mkdir -p checkpoints
!mkdir img

In [ ]:
T_MAX = 20
ALPHA = 0

def rk4(t0, y0, z0, t_max, h): # Explicit Runge-Kutta of order 4
    def __g(x, y, z):
        return -(x * z + (x ** 2 - ALPHA ** 2) * y)/ (x ** 2)
    def __f(x, y, z):
        return z
    def __next(x0, y0, z0):
        k0 = h * __f(x0, y0, z0)
        m0 = h * __g(x0, y0, z0)

        k1 = h * __f(x0 + h/2, y0 + k0/2, z0 + m0/2)
        m1 = h * __g(x0 + h/2, y0 + k0/2, z0 + m0/2)

        k2 = h * __f(x0 + h/2, y0 + k1/2, z0 + m1/2)
        m2 = h * __g(x0 + h/2, y0 + k1/2, z0 + m1/2)

        k3 = h * __f(x0 + h, y0 + k2, z0 + m2)
        m3 = h * __g(x0 + h, y0 + k2, z0 + m2)

        y = y0 + (k0 + 2*k1 + 2*k2 + k3)/6
        z = z0 + (m0 + 2*m1 + 2*m2 + m3)/6
        
        return y, z
    
    x = np.arange(t0, t_max, h)
    y = np.zeros_like(x)
    y[0] = y0
    z = z0
    
    for i in range(1, len(x)):
        y[i], z = __next(x[i-1], y[i-1], z)

    return x, y

t0 = 0.01
y0 = j0(t0)
z0 = jvp(v=ALPHA, z=t0, n=1)
x_rk4, y_rk4 = rk4(t0=t0, y0=y0, z0=z0, t_max=T_MAX, h=0.1)

ref_x = np.linspace(0., 20., 100) # Segment we're calculating mae on
y_true = j0(x_rk4) 
rk4_mae = mean_absolute_error(y_true, y_rk4)

In [ ]:
x_real = np.linspace(0., 20., 100)
y_real = j0(x_real)
plt.grid()
plt.plot(x_real, y_real, label="scipy", color="black")
plt.plot(x_rk4, y_rk4, linestyle="dashed", color="lime", label="rk4", linewidth=1)
plt.title("Runge-Kutta of Order 4")
plt.legend()
plt.show()

In [ ]:
frames = []

def create_frame(t, model):
    global frames
    x_real = np.linspace(0., 20., 100)
    y_real = j0(x_real)
    plt.ylim(-0.5, 1.2)
    plt.grid()
    plt.plot(x_real, y_real)
    plt.plot(x_real, model(x_real))
    plt.title(f'Epoch {t}',
              fontsize=14)
    plt.savefig(f'./img/img_{t}.png', 
                transparent = False,  
                facecolor = 'white'
               )
    frames.append(imageio.imread(f"./img/img_{t}.png"))
    plt.close()

In [9]:
# Training

# Constants
EPOCHS = 10_000 # Number of epochs

LOG_FREQ = 100 # Print results every N epochs
CHECKPOINT_FREQ = 35_000 # Create checkpoint every N epochs
FRAMERATE = 500 # Save image every N epochs
LOSS_FREQ = 500 # Save loss every N epochs

PHYS_COEFF = 1e-3 # Physics loss coeff

# Creating loss graph
loss_x = np.linspace(0, EPOCHS + 1, EPOCHS // LOSS_FREQ + 1) # Epoch number
loss_y = np.zeros_like(loss_x) # Loss value

# MAE graph
pinn_mae_x = np.zeros(EPOCHS // LOSS_FREQ + 1)
pinn_mae_y = np.zeros_like(pinn_mae_x) # Array of mae over the epochs
ref_x = np.linspace(0., 20., 100) # Segment we're calculating mae on
y_true = j0(ref_x) 

opt = tf.keras.optimizers.Adam(learning_rate=1e-3) # Optimizer

for i in range(EPOCHS + 1): # Main loop
    with tf.GradientTape(persistent=True) as tape: # Evaluates first derivative
        with tf.GradientTape() as tape1:
            y_0 = model(x_bound, training=True)
            dy_0 = tape1.gradient(y_0, x_bound)
            bc_1 = tf.square(dy_0) # Boundary derivative condition

        with tf.GradientTape() as tape2:
            y_0 = model(x_phys, training=True)
            dy_0 = tape2.gradient(y_0, x_phys)
            d2y_0 = tape.gradient(dy_0, x_phys)
            phys_loss = tf.reduce_mean(tf.square(x_phys ** 2 * d2y_0 + x_phys * dy_0 + x_phys ** 2 * y_0), axis=0) # Physics loss

        y_0 = model(x_bound)
        bound_loss = tf.square(y_0 - 1.) # Boundary value condition
        
        y = model(x_inn)
        inn_loss = tf.reduce_mean(tf.square(y - y_inn), axis=0)
        
        loss = bound_loss + bc_1 + PHYS_COEFF * phys_loss + inn_loss # Loss
        
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
        
    del tape

    
    
    if i % FRAMERATE == 0:
        create_frame(i, model)
    if i % CHECKPOINT_FREQ == 0:
        model.save(f"checkpoints/checkpoint_epoch_{i}.h5")
        # display(FileLink(rf"checkpoints/checkpoint_epoch_{i}.h5"))
    if i % LOSS_FREQ == 0:
        loss_y[i // LOSS_FREQ] = loss
        cur_mae = mean_absolute_error(y_true, model(ref_x))
        pinn_mae_x[i // LOSS_FREQ] = i
        pinn_mae_y[i // LOSS_FREQ] = cur_mae
        if cur_mae <= 1e-6:
            break
    if i % LOG_FREQ == 0:
        print(f"Epoch {i}, loss: {loss}")
    

NameError: name 'x_phys' is not defined

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))

ax[0].grid()
ax[0].set_yscale("log")
ax[0].set_ylim(1e-5, 1e2)
ax[0].plot(loss_x, loss_y, label="Loss")
ax[0].set_ylabel("Loss value")
ax[0].set_xlabel("Epochs")

ax[1].grid()
ax[1].set_yscale("log")
ax[1].set_ylim(1e-3, 1)
ax[1].plot(pinn_mae_x, pinn_mae_y, label="MAE")
ax[1].plot([pinn_mae_x[0], pinn_mae_x[-1]], [rk4_mae, rk4_mae], linestyle="dashed")
ax[1].set_ylabel("MAE value")
ax[1].set_xlabel("Epochs")
plt.show()

In [ ]:
x_real = np.linspace(0., 20., 100)
y_real = j0(x_real)
plt.grid()
plt.plot(x_real, y_real)
plt.plot(x_real, model(x_real))
plt.scatter(x_phys, [0] * x_phys.shape[0], color="green", s=1, zorder=3)
plt.scatter(x_bound, j0(x_bound), color="red", s=3, zorder=3)
plt.scatter(x_inn, j0(x_inn), color="magenta", s=3, zorder=3)
plt.title("Bessel's function of order 0")
plt.show()

In [ ]:
# Saving GIF
imageio.mimsave('./example.gif', # output gif
                frames,          # array of input frames
                fps=5,
                loop=0)         # optional: frames per second

In [ ]:
# Saving model
!mkdir -p saved_model
model.save('my_model.h5')
FileLink(r'my_model.h5')

![learning](./example.gif)